<a href="https://colab.research.google.com/github/minmincg/house_pricing_analysis/blob/main/Notebooks/neural_networks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import tensorflow as tf

#Import the csv by using the raw link
hpricing_df_og = pd.read_csv("https://raw.githubusercontent.com/minmincg/house_pricing_analysis/24b41d230570324f8d64a1326bcc9e9972a62608/Resources/austin_housing_reduced.csv")
hpricing_df_og.head()

,city,streetAddress,zipcode,latitude,longitude,propertyTaxRate,garageSpaces,hasCooling,hasGarage,hasHeating,...,numOfWaterfrontFeatures,numOfWindowFeatures,numOfCommunityFeatures,lotSizeSqFt,livingAreaSqFt,avgSchoolRating,numOfBathrooms,numOfBedrooms,numOfStories,numOfSchools
0,pflugerville,14424 Lake Victor Dr,78660,30.430632,-97.663078,1.98,2,True,True,True,...,0,1,0,6011.0,2601,2.666667,3.0,4,2,3
1,pflugerville,1104 Strickling Dr,78660,30.432672,-97.661697,1.98,2,True,True,True,...,0,0,0,6185.0,1768,2.666667,2.0,4,1,3
2,pflugerville,1408 Fort Dessau Rd,78660,30.409748,-97.639771,1.98,0,True,False,True,...,0,0,0,7840.0,1478,3.000000,2.0,3,1,4
3,pflugerville,1025 Strickling Dr,78660,30.432112,-97.661659,1.98,2,True,True,True,...,0,0,0,6098.0,1678,2.666667,2.0,3,1,3
4,pflugerville,15005 Donna Jane Loop,78660,30.437368,-97.656860,1.98,0,True,False,True,...,0,0,0,6708.0,2132,4.000000,3.0,3,2,3


In [2]:
# Determine the number of unique values in each column.
hpricing_df_og.nunique()

city                              9
streetAddress                 15000
zipcode                          48
latitude                      14269
longitude                     12379
propertyTaxRate                   3
garageSpaces                     14
hasCooling                        2
hasGarage                         2
hasHeating                        2
hasSpa                            2
hasView                           2
homeType                         10
yearBuilt                       115
latestPrice                    2338
numPriceChanges                  22
numOfAccessibilityFeatures        8
numOfAppliances                  13
numOfParkingFeatures              7
numOfPatioAndPorchFeatures        9
numOfSecurityFeatures             7
numOfWaterfrontFeatures           3
numOfWindowFeatures               5
numOfCommunityFeatures            9
lotSizeSqFt                    1403
livingAreaSqFt                 3616
avgSchoolRating                  27
numOfBathrooms              

In [3]:
column="numOfCommunityFeatures"
hpricing_df_og[column].value_counts()

0    14800
1      178
2       11
3        5
5        4
4        4
6        3
7        1
8        1
Name: numOfCommunityFeatures, dtype: int64

In [4]:
# Drop the non-beneficial ID columns, 'city' and 'streetAddress'.
hpricing_df = hpricing_df_og.drop(columns = ['city', 
                                             'streetAddress',
#                                              'latitude',
#                                              'longitude',
#                                              'numPriceChanges',
                                             'hasGarage',
                                             'homeType',
#                                              'zipcode',
#                                              'yearBuilt',
                                             'numOfAppliances',
                                             'avgSchoolRating',
                                             'numOfSchools'
                                            ], axis=1)
hpricing_df.head()

,zipcode,latitude,longitude,propertyTaxRate,garageSpaces,hasCooling,hasHeating,hasSpa,hasView,yearBuilt,...,numOfPatioAndPorchFeatures,numOfSecurityFeatures,numOfWaterfrontFeatures,numOfWindowFeatures,numOfCommunityFeatures,lotSizeSqFt,livingAreaSqFt,numOfBathrooms,numOfBedrooms,numOfStories
0,78660,30.430632,-97.663078,1.98,2,True,True,False,False,2012,...,1,3,0,1,0,6011.0,2601,3.0,4,2
1,78660,30.432672,-97.661697,1.98,2,True,True,False,False,2013,...,0,0,0,0,0,6185.0,1768,2.0,4,1
2,78660,30.409748,-97.639771,1.98,0,True,True,False,False,2018,...,0,1,0,0,0,7840.0,1478,2.0,3,1
3,78660,30.432112,-97.661659,1.98,2,True,True,False,False,2013,...,0,0,0,0,0,6098.0,1678,2.0,3,1
4,78660,30.437368,-97.656860,1.98,0,True,True,False,False,2002,...,0,0,0,0,0,6708.0,2132,3.0,3,2


In [5]:
# Making sure the columns got dropped
hpricing_df.columns

Index(['zipcode', 'latitude', 'longitude', 'propertyTaxRate', 'garageSpaces',
       'hasCooling', 'hasHeating', 'hasSpa', 'hasView', 'yearBuilt',
       'latestPrice', 'numPriceChanges', 'numOfAccessibilityFeatures',
       'numOfParkingFeatures', 'numOfPatioAndPorchFeatures',
       'numOfSecurityFeatures', 'numOfWaterfrontFeatures',
       'numOfWindowFeatures', 'numOfCommunityFeatures', 'lotSizeSqFt',
       'livingAreaSqFt', 'numOfBathrooms', 'numOfBedrooms', 'numOfStories'],
      dtype='object')

In [6]:
hpricing_df["isAccessible"]=hpricing_df["numOfAccessibilityFeatures"].map(lambda x:(1 if x>0 else 0))
hpricing_df["hasWaterfront"]=hpricing_df["numOfWaterfrontFeatures"].map(lambda x:(1 if x>0 else 0))
# hpricing_df["hasCommunityFeatures"]=hpricing_df["numOfCommunityFeatures"].map(lambda x:(1 if x>0 else 0))
hpricing_df=hpricing_df.drop([
                              "numOfAccessibilityFeatures",
                              "numOfWaterfrontFeatures",
                              "numOfCommunityFeatures"
                             ],
                             axis=1)

In [7]:
# Look at 'zipcode' value counts for binning
zip_counts = hpricing_df["zipcode"].value_counts()
zip_counts

78748    1137
78745    1025
78749     772
78704     693
78737     611
78739     603
78732     590
78759     559
78723     554
78757     518
78717     515
78731     487
78744     469
78753     438
78727     414
78754     393
78758     393
78702     382
78747     381
78750     321
78729     317
78735     308
78703     294
78721     244
78730     243
78728     224
78724     223
78741     217
78733     202
78746     190
78726     178
78751     172
78725     162
78736     162
78752     151
78756     137
78722      96
78617      85
78705      64
78660      36
78619      12
78734       9
78701       8
78719       5
78652       4
78653       4
78742       4
78738       1
Name: zipcode, dtype: int64

In [8]:
# Choose a cutoff value of less than 200 and create a list of application types to be replaced

zipcodes_to_replace = list(zip_counts[zip_counts<200].index)

# Replace in dataframe
for zip in zipcodes_to_replace:
    hpricing_df['zipcode'] = hpricing_df['zipcode'].replace(zip,"Other")

# Making sure the binning was successful
hpricing_df['zipcode'].value_counts()

Other    1480
78748    1137
78745    1025
78749     772
78704     693
78737     611
78739     603
78732     590
78759     559
78723     554
78757     518
78717     515
78731     487
78744     469
78753     438
78727     414
78754     393
78758     393
78702     382
78747     381
78750     321
78729     317
78735     308
78703     294
78721     244
78730     243
78728     224
78724     223
78741     217
78733     202
Name: zipcode, dtype: int64

In [9]:
# hometype_counts=hpricing_df['homeType'].value_counts()

# hometype_to_replace = list(hometype_counts[hometype_counts<50].index)

# # Replace in dataframe
# for hometype in hometype_to_replace:
#     hpricing_df['homeType'] = hpricing_df['homeType'].replace(hometype,"Other")

# # Making sure the binning was successful
# hpricing_df['homeType'].value_counts()

In [10]:
#Check data types
hpricing_df.dtypes

zipcode                        object
latitude                      float64
longitude                     float64
propertyTaxRate               float64
garageSpaces                    int64
hasCooling                       bool
hasHeating                       bool
hasSpa                           bool
hasView                          bool
yearBuilt                       int64
latestPrice                     int64
numPriceChanges                 int64
numOfParkingFeatures            int64
numOfPatioAndPorchFeatures      int64
numOfSecurityFeatures           int64
numOfWindowFeatures             int64
lotSizeSqFt                   float64
livingAreaSqFt                  int64
numOfBathrooms                float64
numOfBedrooms                   int64
numOfStories                    int64
isAccessible                    int64
hasWaterfront                   int64
dtype: object

In [11]:
#Convert zipcode to dummies because it is a categorical
#Didnt use dummies in the rest of the data (hasCooling, hasGarage, hasHeating, hasSpa, hasView, homeType) because it was already numerical
hpricing_df = pd.get_dummies(hpricing_df,dtype=float)
hpricing_df

#hpricing_df['zipcode']= hpricing_df['zipcode'].astype(float)
#hpricing_df.dtypes

,latitude,longitude,propertyTaxRate,garageSpaces,hasCooling,hasHeating,hasSpa,hasView,yearBuilt,latestPrice,...,zipcode_78747,zipcode_78748,zipcode_78749,zipcode_78750,zipcode_78753,zipcode_78754,zipcode_78757,zipcode_78758,zipcode_78759,zipcode_Other
0,30.430632,-97.663078,1.98,2,True,True,False,False,2012,305000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,30.432672,-97.661697,1.98,2,True,True,False,False,2013,295000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
2,30.409748,-97.639771,1.98,0,True,True,False,False,2018,256125,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,30.432112,-97.661659,1.98,2,True,True,False,False,2013,240000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,30.437368,-97.656860,1.98,0,True,True,False,False,2002,239900,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15002,30.161997,-97.816345,1.98,6,True,True,False,True,1986,330000,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15003,30.240223,-97.772835,1.98,2,True,True,True,True,2016,550000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15004,30.244835,-97.761124,1.98,2,True,True,False,False,1936,875000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
15005,30.281569,-97.717789,1.98,0,True,True,False,False,1968,420000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [12]:
hpricing_df.columns

Index(['latitude', 'longitude', 'propertyTaxRate', 'garageSpaces',
       'hasCooling', 'hasHeating', 'hasSpa', 'hasView', 'yearBuilt',
       'latestPrice', 'numPriceChanges', 'numOfParkingFeatures',
       'numOfPatioAndPorchFeatures', 'numOfSecurityFeatures',
       'numOfWindowFeatures', 'lotSizeSqFt', 'livingAreaSqFt',
       'numOfBathrooms', 'numOfBedrooms', 'numOfStories', 'isAccessible',
       'hasWaterfront', 'zipcode_78702', 'zipcode_78703', 'zipcode_78704',
       'zipcode_78717', 'zipcode_78721', 'zipcode_78723', 'zipcode_78724',
       'zipcode_78727', 'zipcode_78728', 'zipcode_78729', 'zipcode_78730',
       'zipcode_78731', 'zipcode_78732', 'zipcode_78733', 'zipcode_78735',
       'zipcode_78737', 'zipcode_78739', 'zipcode_78741', 'zipcode_78744',
       'zipcode_78745', 'zipcode_78747', 'zipcode_78748', 'zipcode_78749',
       'zipcode_78750', 'zipcode_78753', 'zipcode_78754', 'zipcode_78757',
       'zipcode_78758', 'zipcode_78759', 'zipcode_Other'],
      dtype='obj

In [13]:
# Split our preprocessed data into our features and target arrays
y= hpricing_df['latestPrice'].values

X= hpricing_df.drop ('latestPrice', axis=1).values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X,y,random_state = 42)

In [14]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [15]:
# Define the model - deep neural net, i.e., the number of input features and hidden nodes for each layer.
number_input_features = len(X_train[0])
hidden_nodes_layer1 =  80
hidden_nodes_layer2 = 30

nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(
    tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu")
)

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="relu"))

# Check the structure of the model
nn.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 80)                4160      
                                                                 
 dense_1 (Dense)             (None, 30)                2430      
                                                                 
 dense_2 (Dense)             (None, 1)                 31        
                                                                 
Total params: 6,621
Trainable params: 6,621
Non-trainable params: 0
_________________________________________________________________


In [16]:
# Compile the model
#mae = mean absolute error & mse = mean squared error
nn.compile(loss="mse", optimizer="adam", metrics=["mae", "mse"])

In [17]:
# Train the model
fit_model = nn.fit(X_train_scaled,y_train,epochs=500)

Epoch 1/500
352/352 [==============================] - 5s 5ms/step - loss: 418697248768.0000 - mae: 508255.1562 - mse: 418697248768.0000
Epoch 2/500
352/352 [==============================] - 2s 5ms/step - loss: 414170611712.0000 - mae: 505305.9375 - mse: 414170611712.0000
Epoch 3/500
352/352 [==============================] - 2s 5ms/step - loss: 398364573696.0000 - mae: 494908.8438 - mse: 398364573696.0000
Epoch 4/500
352/352 [==============================] - 2s 5ms/step - loss: 368175808512.0000 - mae: 474374.8125 - mse: 368175808512.0000
Epoch 5/500
352/352 [==============================] - 2s 6ms/step - loss: 325322506240.0000 - mae: 442819.9062 - mse: 325322506240.0000
Epoch 6/500
352/352 [==============================] - 2s 6ms/step - loss: 276543963136.0000 - mae: 402016.6562 - mse: 276543963136.0000
Epoch 7/500
352/352 [==============================] - 2s 7ms/step - loss: 229389844480.0000 - mae: 357634.5312 - mse: 229389844480.0000
Epoch 8/500
352/352 [====================

352/352 [==============================] - 2s 5ms/step - loss: 52054851584.0000 - mae: 118346.5469 - mse: 52054851584.0000
Epoch 62/500
352/352 [==============================] - 2s 5ms/step - loss: 51903778816.0000 - mae: 118212.0000 - mse: 51903778816.0000
Epoch 63/500
352/352 [==============================] - 2s 5ms/step - loss: 51761184768.0000 - mae: 117918.0391 - mse: 51761184768.0000
Epoch 64/500
352/352 [==============================] - 2s 5ms/step - loss: 51602935808.0000 - mae: 117939.2031 - mse: 51602935808.0000
Epoch 65/500
352/352 [==============================] - 2s 5ms/step - loss: 51471949824.0000 - mae: 117224.8594 - mse: 51471949824.0000
Epoch 66/500
352/352 [==============================] - 2s 6ms/step - loss: 51345788928.0000 - mae: 117452.4844 - mse: 51345788928.0000
Epoch 67/500
352/352 [==============================] - 2s 5ms/step - loss: 51201945600.0000 - mae: 116895.3750 - mse: 51201945600.0000
Epoch 68/500
352/352 [==============================] - 2s 5m

352/352 [==============================] - 2s 6ms/step - loss: 47205036032.0000 - mae: 111334.3125 - mse: 47205036032.0000
Epoch 122/500
352/352 [==============================] - 2s 6ms/step - loss: 47151648768.0000 - mae: 111247.4531 - mse: 47151648768.0000
Epoch 123/500
352/352 [==============================] - 2s 6ms/step - loss: 47106691072.0000 - mae: 111087.0078 - mse: 47106691072.0000
Epoch 124/500
352/352 [==============================] - 2s 6ms/step - loss: 47052414976.0000 - mae: 111056.6875 - mse: 47052414976.0000
Epoch 125/500
352/352 [==============================] - 2s 6ms/step - loss: 47022059520.0000 - mae: 110725.7891 - mse: 47022059520.0000
Epoch 126/500
352/352 [==============================] - 2s 6ms/step - loss: 46972956672.0000 - mae: 111028.7344 - mse: 46972956672.0000
Epoch 127/500
352/352 [==============================] - 2s 6ms/step - loss: 46917394432.0000 - mae: 110864.4297 - mse: 46917394432.0000
Epoch 128/500
352/352 [==============================] 

352/352 [==============================] - 2s 5ms/step - loss: 44837654528.0000 - mae: 106735.1875 - mse: 44837654528.0000
Epoch 181/500
352/352 [==============================] - 2s 5ms/step - loss: 44826054656.0000 - mae: 106411.7656 - mse: 44826054656.0000
Epoch 182/500
352/352 [==============================] - 2s 6ms/step - loss: 44783603712.0000 - mae: 106538.0000 - mse: 44783603712.0000
Epoch 183/500
352/352 [==============================] - 2s 5ms/step - loss: 44761640960.0000 - mae: 106360.6953 - mse: 44761640960.0000
Epoch 184/500
352/352 [==============================] - 2s 5ms/step - loss: 44728193024.0000 - mae: 106228.6172 - mse: 44728193024.0000
Epoch 185/500
352/352 [==============================] - 2s 6ms/step - loss: 44693274624.0000 - mae: 106514.0469 - mse: 44693274624.0000
Epoch 186/500
352/352 [==============================] - 2s 5ms/step - loss: 44670910464.0000 - mae: 106136.8516 - mse: 44670910464.0000
Epoch 187/500
352/352 [==============================] 

352/352 [==============================] - 2s 5ms/step - loss: 43278172160.0000 - mae: 103238.2969 - mse: 43278172160.0000
Epoch 240/500
352/352 [==============================] - 2s 5ms/step - loss: 43240951808.0000 - mae: 103132.1406 - mse: 43240951808.0000
Epoch 241/500
352/352 [==============================] - 2s 5ms/step - loss: 43235864576.0000 - mae: 103198.8750 - mse: 43235864576.0000
Epoch 242/500
352/352 [==============================] - 2s 6ms/step - loss: 43217981440.0000 - mae: 103079.2812 - mse: 43217981440.0000
Epoch 243/500
352/352 [==============================] - 2s 5ms/step - loss: 43188006912.0000 - mae: 102989.8594 - mse: 43188006912.0000
Epoch 244/500
352/352 [==============================] - 2s 5ms/step - loss: 43163459584.0000 - mae: 103138.0078 - mse: 43163459584.0000
Epoch 245/500
352/352 [==============================] - 2s 5ms/step - loss: 43149148160.0000 - mae: 102807.9531 - mse: 43149148160.0000
Epoch 246/500
352/352 [==============================] 

352/352 [==============================] - 2s 6ms/step - loss: 42225393664.0000 - mae: 101024.8125 - mse: 42225393664.0000
Epoch 299/500
352/352 [==============================] - 2s 6ms/step - loss: 42210766848.0000 - mae: 101042.3438 - mse: 42210766848.0000
Epoch 300/500
352/352 [==============================] - 2s 5ms/step - loss: 42192375808.0000 - mae: 101006.6719 - mse: 42192375808.0000
Epoch 301/500
352/352 [==============================] - 2s 6ms/step - loss: 42182889472.0000 - mae: 101028.9609 - mse: 42182889472.0000
Epoch 302/500
352/352 [==============================] - 2s 6ms/step - loss: 42147827712.0000 - mae: 100748.0078 - mse: 42147827712.0000
Epoch 303/500
352/352 [==============================] - 2s 6ms/step - loss: 42157449216.0000 - mae: 101257.6016 - mse: 42157449216.0000
Epoch 304/500
352/352 [==============================] - 2s 6ms/step - loss: 42138841088.0000 - mae: 100870.5703 - mse: 42138841088.0000
Epoch 305/500
352/352 [==============================] 

352/352 [==============================] - 2s 6ms/step - loss: 41477283840.0000 - mae: 99791.3359 - mse: 41477283840.0000
Epoch 359/500
352/352 [==============================] - 2s 6ms/step - loss: 41462321152.0000 - mae: 99773.6250 - mse: 41462321152.0000
Epoch 360/500
352/352 [==============================] - 2s 6ms/step - loss: 41452351488.0000 - mae: 99604.8906 - mse: 41452351488.0000
Epoch 361/500
352/352 [==============================] - 2s 6ms/step - loss: 41449644032.0000 - mae: 99782.3594 - mse: 41449644032.0000
Epoch 362/500
352/352 [==============================] - 2s 6ms/step - loss: 41416884224.0000 - mae: 99805.2422 - mse: 41416884224.0000
Epoch 363/500
352/352 [==============================] - 2s 6ms/step - loss: 41422467072.0000 - mae: 99494.4375 - mse: 41422467072.0000
Epoch 364/500
352/352 [==============================] - 2s 6ms/step - loss: 41410523136.0000 - mae: 99720.8125 - mse: 41410523136.0000
Epoch 365/500
352/352 [==============================] - 2s 6m

352/352 [==============================] - 2s 4ms/step - loss: 40888664064.0000 - mae: 98730.3438 - mse: 40888664064.0000
Epoch 419/500
352/352 [==============================] - 2s 4ms/step - loss: 40875302912.0000 - mae: 98766.2578 - mse: 40875302912.0000
Epoch 420/500
352/352 [==============================] - 2s 4ms/step - loss: 40865288192.0000 - mae: 98528.3047 - mse: 40865288192.0000
Epoch 421/500
352/352 [==============================] - 2s 4ms/step - loss: 40862433280.0000 - mae: 98789.0703 - mse: 40862433280.0000
Epoch 422/500
352/352 [==============================] - 1s 4ms/step - loss: 40846868480.0000 - mae: 98565.9375 - mse: 40846868480.0000
Epoch 423/500
352/352 [==============================] - 2s 4ms/step - loss: 40843984896.0000 - mae: 98776.6016 - mse: 40843984896.0000
Epoch 424/500
352/352 [==============================] - 1s 4ms/step - loss: 40826388480.0000 - mae: 98592.3906 - mse: 40826388480.0000
Epoch 425/500
352/352 [==============================] - 2s 4m

352/352 [==============================] - 2s 4ms/step - loss: 40356954112.0000 - mae: 97775.7734 - mse: 40356954112.0000
Epoch 479/500
352/352 [==============================] - 2s 4ms/step - loss: 40337907712.0000 - mae: 97833.5078 - mse: 40337907712.0000
Epoch 480/500
352/352 [==============================] - 2s 4ms/step - loss: 40348147712.0000 - mae: 98005.6953 - mse: 40348147712.0000
Epoch 481/500
352/352 [==============================] - 2s 4ms/step - loss: 40320339968.0000 - mae: 97901.5078 - mse: 40320339968.0000
Epoch 482/500
352/352 [==============================] - 2s 4ms/step - loss: 40321609728.0000 - mae: 97906.3750 - mse: 40321609728.0000
Epoch 483/500
352/352 [==============================] - 2s 4ms/step - loss: 40313929728.0000 - mae: 97821.0156 - mse: 40313929728.0000
Epoch 484/500
352/352 [==============================] - 2s 5ms/step - loss: 40302518272.0000 - mae: 97703.2344 - mse: 40302518272.0000
Epoch 485/500
352/352 [==============================] - 2s 4m

In [18]:
# Evaluate the model using the test data
errors = nn.evaluate(X_test_scaled,y_test,verbose=2)
errors
print(f"MSE: {errors [0]}, MAE: {errors[1]}")

118/118 - 1s - loss: 38402551808.0000 - mae: 99770.9609 - mse: 38402551808.0000 - 842ms/epoch - 7ms/step
MSE: 38402551808.0, MAE: 99770.9609375


In [19]:
# Evaluate the model using the test data
errors = nn.evaluate(X_test_scaled,y_test,verbose=2)
errors
print(f"MSE: {errors [0]}, MAE: {errors[1]}")

118/118 - 0s - loss: 38402551808.0000 - mae: 99770.9609 - mse: 38402551808.0000 - 405ms/epoch - 3ms/step
MSE: 38402551808.0, MAE: 99770.9609375


In [20]:
nn.predict([X_train_scaled.tolist()[1]])


1/1 [==============================] - 0s 313ms/step


array([[570063.9]], dtype=float32)

In [31]:
# Export our model to HDF5 file
nn.save("../Resources/nn/neural_network.h5")

In [30]:
# Export standardScaler from sklearn to file
from joblib import dump, load
dump(scaler, '../Resources/nn/std_scaler.bin', compress=True)

['../Resources/nn/std_scaler.bin']

## Predict prices for original table

In [23]:
X_scaled=scaler.transform(X)
predict_price_nn=nn.predict(X_scaled).flatten()
predict_price_nn

469/469 [==============================] - 2s 3ms/step


array([ 524708.44,  293793.25,  298402.62, ...,  667760.75, 1734900.6 ,
        371365.3 ], dtype=float32)

In [24]:
hpricing_extra_nn=pd.DataFrame()
hpricing_extra_nn["predict_price_nn"]=predict_price_nn
hpricing_extra_nn["latestPrice"]=hpricing_df_og["latestPrice"]
hpricing_extra_nn["difference_nn"]=hpricing_extra_nn["latestPrice"]-hpricing_extra_nn["predict_price_nn"]
hpricing_extra_nn["percent_change_nn"]=round(hpricing_extra_nn["difference_nn"]*100/hpricing_extra_nn["latestPrice"],2)
hpricing_extra_nn.head(10)

,predict_price_nn,latestPrice,difference_nn,percent_change_nn
0,524708.43750,305000,-219708.43750,-72.04
1,293793.25000,295000,1206.75000,0.41
2,298402.62500,256125,-42277.62500,-16.51
3,267403.84375,240000,-27403.84375,-11.42
4,371112.31250,239900,-131212.31250,-54.69
5,338855.65625,309045,-29810.65625,-9.65
6,458584.43750,315000,-143584.43750,-45.58
7,233349.81250,219900,-13449.81250,-6.12
8,321599.84375,225000,-96599.84375,-42.93
9,249701.00000,194800,-54901.00000,-28.18


In [25]:
hpricing_extra_nn.drop(columns=["latestPrice"],inplace=True)
hpricing_extra_nn.head(5)

,predict_price_nn,difference_nn,percent_change_nn
0,524708.43750,-219708.43750,-72.04
1,293793.25000,1206.75000,0.41
2,298402.62500,-42277.62500,-16.51
3,267403.84375,-27403.84375,-11.42
4,371112.31250,-131212.31250,-54.69


In [28]:
hpricing_extra_nn.to_csv("../Resources/austin_extra_nn.csv",index=False)

In [29]:
hpricing_extra_nn.mean()

predict_price_nn     509919.039211
difference_nn         -1472.489068
percent_change_nn        -7.401035
dtype: float64